In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from llama import Workflow, Llama
from llama.util import find_free_port

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = str(find_free_port())

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8*8192,
    max_batch_size=1,
    model_parallel_size=1,
    max_nodes=100,
)

llama = Llama(workflow.model, workflow.tokenizer)

/home/tbai4/llama3/llama/util.py:60: UserWarning: 65536 does not lie within [1, 8192]
  warnings.warn(f"{max_seq_len} does not lie within [1, 8192]")


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 64.75 seconds


In [ ]:
correct = []

In [6]:
import json
from llama.workflows.madpar import eval_debate_solutions, parse_output
from llama.workflows.tot import load_math_problems

llama.model.reshape_cache(4)
problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='test')[:500]

for path in [
    # '/home/tbai4/llama3/dumps/madpar/math_baseline_preft_test.json',
    # '/home/tbai4/llama3/dumps/madpar/math_cached_preft.json',
    '/home/tbai4/llama3/dumps/madpar/math_cached_postft_lora_step-2699.pt.json',
]:
    with open(path) as f:
        data = json.load(f)
        print(len(data))

    outputs = eval_debate_solutions(
        llama,
        agent_solutions=[
            [parse_output(llama.tokenizer.decode(a)) for a in d['outputs']['debate_tokens'][-1]]
            for d in data
        ],
        problems=problems[:len(data)],
    )
    correct.append(outputs)
    
    with open(f'{path}_correct.json', 'w') as f:
        json.dump(outputs, f)

    print(sum(outputs))

500


100%|██████████| 500/500 [14:06<00:00,  1.69s/it]

300


In [4]:
import json
from llama.workflows.madpar import parse_output, eval_debate_solutions
from llama.workflows.tot import load_math_problems

llama.model.reshape_cache(4)

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='test')
p2s = {d['problem']: d['solution'] for d in problems}

for path in [
    '/home/tbai4/llama3/dumps/madpar/math_baseline_preft_test.json',
    '/home/tbai4/llama3/dumps/madpar/math_cached_preft_test.json',
]:

    with open(path) as f:
        data = json.load(f)

    correct = eval_debate_solutions(
        llama,
        agent_solutions=[
            [parse_output(llama.tokenizer.decode(a)) for a in d['outputs']['debate_tokens'][-1]]
            for d in data
        ],
        problems=[{
            'problem': d['inputs']['problem'],
            'solution': p2s[d['inputs']['problem']]
        } for d in data]
    )

    print(sum(correct), len(correct))

100%|██████████| 500/500 [20:25<00:00,  2.45s/it]


323 500


100%|██████████| 360/360 [23:30<00:00,  3.92s/it]

188 360


In [ ]:
from llama.workflows.madpar import madpar_baseline
from llama.workflows.tot import load_math_problems

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='val')

outputs = madpar_baseline(
    workflow=workflow,
    problem=problems[0]['problem'],
    num_agents=3,
    num_rounds=3,
    debug=True,
)

In [3]:
# sample = {
#     'inputs': {'problem': problems[0]['problem']},
#     'outputs': outputs
# }

# import json
# with open('tmp', 'w') as f:
#     json.dump(sample, f)
    
import json
with open('tmp', 'r') as f:
    sample = json.load(f)

In [4]:
from llama.workflows.trainers.madpar import MadparTrainer

trainer = MadparTrainer(workflow, '/scratch4/jeisner1/tjbai/checkpoints/madpar', 3e-4)

Training 54.5M / 8.1B parameters


In [ ]:
from tqdm import tqdm
for i in tqdm(range(100)):
    trainer.optimizer.zero_grad()
    loss, metrics = trainer.step(sample)
    loss.backward()
    print(metrics['train/total_loss'])
    trainer.optimizer.step()

In [ ]:
from llama.workflows.madpar import mad_cached

outputs = mad_cached(
    workflow=workflow,
    problem=problems[0]['problem'],
    num_agents=3,
    num_rounds=3,
    debug=True
)

In [9]:
import json
from llama.workflows.tot import eval_solutions, load_math_problems

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='train')[:500]
print(len(problems))

with open('/home/tbai4/llama3/dumps/madpar/math_baseline_e2e.json') as f:
    data = json.load(f)

500


In [10]:
print(sum(
    any(a is None for a in d['outputs']['final_answers'])
    for d in data
))

62


In [25]:
data[8]['outputs']['final_answers']

['5/7', '\\frac{5', '5/7']